In [1]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg

StatementMeta(, 399f811a-f437-4419-b23c-3456b5b0ddd5, 5, Finished, Available, Finished)

In [2]:
df = spark.read.table("fabrics_project_earthquake_silver_layer")

StatementMeta(, 399f811a-f437-4419-b23c-3456b5b0ddd5, 6, Finished, Available, Finished)

In [3]:
display(df)

StatementMeta(, 399f811a-f437-4419-b23c-3456b5b0ddd5, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bf41dc32-349b-4319-983e-67367f61e2d1)

In [4]:

def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 399f811a-f437-4419-b23c-3456b5b0ddd5, 8, Finished, Available, Finished)

In [5]:
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 399f811a-f437-4419-b23c-3456b5b0ddd5, 9, Finished, Available, Finished)

In [6]:
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, 399f811a-f437-4419-b23c-3456b5b0ddd5, 10, Finished, Available, Finished)

In [7]:
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(, 399f811a-f437-4419-b23c-3456b5b0ddd5, 11, Finished, Available, Finished)

In [8]:
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, 399f811a-f437-4419-b23c-3456b5b0ddd5, 12, Finished, Available, Finished)